In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
import requests
import json
import pymysql
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
pip install pandas numpy sentence-transformers faiss-cpu requests pymysql

In [ ]:
# Функция для чтения файла и возврата списка строк
def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return [line.strip() for line in file.readlines()]

# Чтение данных из файлов
test_questions = read_file('test.nl')
test_queries = read_file('test.sql')
train_questions = read_file('train.nl')
train_queries = read_file('train.sql')

# Создание датафреймов
test_df = pd.DataFrame({
    'question': test_questions,
    'sql_query': test_queries
})

train_df = pd.DataFrame({
    'question': train_questions,
    'sql_query': train_queries
})

In [3]:
pip install rouge-score -q

Note: you may need to restart the kernel to use updated packages.


In [3]:
table_schema = """
CREATE TABLE restriction (no_discounts BOOLEAN, minimum_stay VARCHAR(50), stopovers VARCHAR(50), restriction_code VARCHAR(50), application VARCHAR(50), maximum_stay VARCHAR(50), saturday_stay_required VARCHAR(50), advance_purchase VARCHAR(50));
CREATE TABLE flight_stop (departure_airline VARCHAR(50), stop_number VARCHAR(50), arrival_flight_number VARCHAR(50), flight_id VARCHAR(50), arrival_time VARCHAR(50), departure_flight_number VARCHAR(50), stop_time VARCHAR(50), arrival_airline VARCHAR(50), stop_days VARCHAR(50), stop_airport VARCHAR(50), departure_time VARCHAR(50));
CREATE TABLE food_service (meal_code VARCHAR(50), compartment VARCHAR(50), meal_number VARCHAR(50), meal_description VARCHAR(50));
CREATE TABLE month (month_number VARCHAR(50), month_name VARCHAR(50));
CREATE TABLE code_description (code VARCHAR(50), description VARCHAR(50));
CREATE TABLE city (city_name VARCHAR(100), country_name VARCHAR(50), state_code VARCHAR(50), time_zone_code VARCHAR(50), city_code VARCHAR(50));
CREATE TABLE flight_fare (flight_id VARCHAR(50), fare_id INT);
CREATE TABLE state (country_name VARCHAR(50), state_code VARCHAR(50), state_name VARCHAR(50));
CREATE TABLE fare_basis (discounted BOOLEAN, class_type VARCHAR(50), season VARCHAR(50), basis_days VARCHAR(50), booking_class VARCHAR(50), night VARCHAR(50), premium VARCHAR(50), fare_basis_code VARCHAR(50), economy VARCHAR(50));
CREATE TABLE date_day (day_number VARCHAR(50), month_number VARCHAR(50), day_name VARCHAR(50), year VARCHAR(50));
CREATE TABLE time_interval (period VARCHAR(50), end_time VARCHAR(50), begin_time VARCHAR(50));
CREATE TABLE flight (to_airport VARCHAR(50), aircraft_code_sequence VARCHAR(50), dual_carrier VARCHAR(50), flight_id VARCHAR(50), stops VARCHAR(50), flight_days VARCHAR(50), connections VARCHAR(50), arrival_time VARCHAR(50), time_elapsed VARCHAR(50), flight_number VARCHAR(50), from_airport VARCHAR(50), airline_flight VARCHAR(50), airline_code VARCHAR(50), meal_code VARCHAR(50), departure_time VARCHAR(50));
CREATE TABLE dual_carrier (low_flight_number VARCHAR(50), high_flight_number VARCHAR(50), main_airline VARCHAR(50), service_name VARCHAR(50), dual_airline VARCHAR(50));
CREATE TABLE aircraft (aircraft_code VARCHAR(50), capacity VARCHAR(50), wing_span VARCHAR(50), engines VARCHAR(50), aircraft_description VARCHAR(50), basic_type VARCHAR(50), weight VARCHAR(50), pressurized VARCHAR(50), length VARCHAR(50), propulsion VARCHAR(50), pay_load VARCHAR(50), cruising_speed VARCHAR(50), range_miles VARCHAR(50), wide_body VARCHAR(50), manufacturer VARCHAR(50));
CREATE TABLE fare (to_airport VARCHAR(50), restriction_code VARCHAR(50), round_trip_required VARCHAR(50), fare_id INT, from_airport VARCHAR(50), one_direction_cost VARCHAR(50), fare_basis_code VARCHAR(50), round_trip_cost VARCHAR(50), fare_airline VARCHAR(50));
CREATE TABLE compartment_class (compartment VARCHAR(50), class_type VARCHAR(50));
CREATE TABLE flight_leg (flight_id VARCHAR(50), leg_number VARCHAR(50), leg_flight VARCHAR(50));
CREATE TABLE days (days_code INT, day_name VARCHAR(50));
CREATE TABLE airport_service (minutes_distant VARCHAR(50), airport_code VARCHAR(50), direction VARCHAR(50), city_code VARCHAR(50), miles_distant VARCHAR(50));
CREATE TABLE airport (airport_code VARCHAR(50), airport_name VARCHAR(100), airport_location VARCHAR(50), minimum_connect_time VARCHAR(50), country_name VARCHAR(50), state_code VARCHAR(50), time_zone_code VARCHAR(50));
CREATE TABLE time_zone (time_zone_name VARCHAR(50), hours_from_gmt VARCHAR(50), time_zone_code VARCHAR(50));
CREATE TABLE airline (note VARCHAR(50), airline_code VARCHAR(50), airline_name VARCHAR(50));
CREATE TABLE equipment_sequence (aircraft_code VARCHAR(50), aircraft_code_sequence VARCHAR(50));
CREATE TABLE ground_service (airport_code VARCHAR(50), transport_type VARCHAR(50), city_code VARCHAR(50), ground_fare VARCHAR(50));
CREATE TABLE class_of_service (booking_class VARCHAR(50), class_description VARCHAR(50), `rank` VARCHAR(50));
);
"""

In [ ]:
# Функция для вызова модели через API
def generate_sql_code(instruction, table_schema, api_key):
    prompt = f"""
    Your task is to write simple, efficient, and accurate SQL queries based on the provided instructions and table schema. Follow these rules:
Answer only with valid SQL code. Do not include any explanations, comments, or additional text.
Keep the query as simple and straightforward as possible.
Use only the columns and tables mentioned in the schema.
Avoid unnecessary joins, subqueries, or complex logic unless explicitly required.

Instruction: {instruction}
Table Schema: {table_schema}
SQL code:
    """
    input_data = {
        "model": "meta-llama/llama-3.2-1b-instruct",
        "messages": [{"role": "user", "content": prompt}]
    }
    response = requests.post(
        url="https://openrouter.ai/api/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json",
        },
        data=json.dumps(input_data)
    )
    if response.status_code == 200:
        data = response.json()
        return data.get('choices', [{}])[0].get('message', {}).get('content', 'No content found')
    else:
        raise RuntimeError(f"API request failed with status {response.status_code}: {response.text}")

# Функция для вычисления метрик
def calculate_metrics(true_answer, model_response):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = scorer.score(true_answer, model_response)

    true_tokens = true_answer.split()
    response_tokens = model_response.split()
    smooth_fn = SmoothingFunction().method1
    bleu_score = sentence_bleu([true_tokens], response_tokens, smoothing_function=smooth_fn)

    meteor = meteor_score([true_answer.split()], model_response.split())

    return bleu_score, meteor, rouge_scores['rouge1'].fmeasure, rouge_scores['rouge2'].fmeasure, rouge_scores['rougeL'].fmeasure

# Функция для выполнения SQL-запроса в MySQL
def execute_mysql_query(sql_query):
    config = {
        'user': '',
        'password': '',
        'host': '',
        'database': '',
        'port': 
    }
    try:
        conn = pymysql.connect(**config)
        cursor = conn.cursor()
        cursor.execute(sql_query)
        rows = cursor.fetchall()
        cursor.close()
        conn.close()
        return {"success": True, "result": rows}
    except Exception as e:
        return {"success": False, "error": str(e)}

# Основной код
def evaluate_model(questions, sql_queries, table_schema, api_key, model_name="SQL Query Generator"):
    metrics_data = []  # Список для хранения строк данных
    successful_queries = 0  # Счётчик успешных запросов

    for i in range(min(50, len(questions))):  # Обрабатываем первые 10 запросов
        instruction = questions[i]
        true_response = sql_queries[i]

        try:
            # Генерация ответа модели
            model_response = generate_sql_code(instruction, table_schema, api_key)

            # Тестируем SQL-код в MySQL
            test_result = execute_mysql_query(model_response)
            sql_valid = test_result["success"]
            sql_error = test_result.get("error", "")

            # Если запрос выполнен успешно, увеличиваем счётчик
            if sql_valid:
                successful_queries += 1

            # Вычисляем метрики
            bleu, meteor, rouge1, rouge2, rougeL = calculate_metrics(true_response, model_response)

            # Добавляем строку данных в список
            metrics_data.append({
                "Instruction": instruction,
                "Response": true_response,
                "Model Response": model_response,
                "BLEU": bleu,
                "METEOR": meteor,
                "ROUGE-1": rouge1,
                "ROUGE-2": rouge2,
                "ROUGE-L": rougeL,
                "SQL Valid": sql_valid,
                "SQL Error": sql_error
            })
        except Exception as e:
            print(f"Error processing query {i}: {e}")

    # Рассчитываем средние значения метрик
    average_bleu = sum(m['BLEU'] for m in metrics_data) / len(metrics_data)
    average_meteor = sum(m['METEOR'] for m in metrics_data) / len(metrics_data)
    average_rouge1 = sum(m['ROUGE-1'] for m in metrics_data) / len(metrics_data)
    average_rouge2 = sum(m['ROUGE-2'] for m in metrics_data) / len(metrics_data)
    average_rougeL = sum(m['ROUGE-L'] for m in metrics_data) / len(metrics_data)

    # Выводим результаты
    print(f"Average BLEU: {average_bleu}")
    print(f"Average METEOR: {average_meteor}")
    print(f"Average ROUGE-1: {average_rouge1}")
    print(f"Average ROUGE-2: {average_rouge2}")
    print(f"Average ROUGE-L: {average_rougeL}")
    print(f"Successful Queries: {successful_queries} out of {len(metrics_data)}")

# Пример использования
OPENROUTER_API_KEY = "sk..."

# Чтение данных из файлов
def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return [line.strip() for line in file.readlines()]

train_questions = read_file('train.nl')
train_queries = read_file('train.sql')

# Оценка модели
evaluate_model(test_questions, test_queries, table_schema, OPENROUTER_API_KEY)

Average BLEU: 0.00725432765893423
Average METEOR: 0.08570991651351278
Average ROUGE-1: 0.2315112046909513
Average ROUGE-2: 0.06288901851253142
Average ROUGE-L: 0.17137393607188806
Successful Queries: 0 out of 50


# RAG

In [ ]:
# Загрузка модели для embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Векторизация вопросов из train_df
train_embeddings = model.encode(train_df['question'].tolist())

# Создание FAISS индекса для быстрого поиска
dimension = train_embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  # Используем Inner Product (эквивалентно косинусной близости)
faiss.normalize_L2(train_embeddings)  # Нормализуем векторы для косинусной близости
index.add(train_embeddings)

# Функция для поиска релевантных запросов
def find_relevant_queries(query, top_k=5):
    query_embedding = model.encode([query])
    faiss.normalize_L2(query_embedding)
    distances, indices = index.search(query_embedding, top_k)
    relevant_queries = train_df.iloc[indices[0]]['sql_query'].tolist()
    return relevant_queries

# Функция для вызова LLM через API
def generate_sql_code(instruction, table_schema, relevant_queries, api_key):
    prompt = f"""
    Answer only with valid SQL code, nothing else. you can use Relevant SQL Queries, as these are real working queries to the database on similar issues.
    Instruction: {instruction}
    Table Schema: {table_schema}
    Relevant SQL Queries: {relevant_queries}
    SQL code:
    """
    input_data = {
        "model": "deepseek/deepseek-prover-v2:free",
        "messages": [{"role": "user", "content": prompt}]
    }
    response = requests.post(
        url="https://openrouter.ai/api/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json",
        },
        data=json.dumps(input_data)
    )
    if response.status_code == 200:
        data = response.json()
        return data.get('choices', [{}])[0].get('message', {}).get('content', 'No content found')
    else:
        raise RuntimeError(f"API request failed with status {response.status_code}: {response.text}")

# Функция для выполнения SQL-запроса в MySQL
def execute_mysql_query(sql_query):
    config = {
        'user': '',
        'password': '',
        'host': '',
        'database': '',
        'port': 
    }
    try:
        conn = pymysql.connect(**config)
        cursor = conn.cursor()
        cursor.execute(sql_query)
        rows = cursor.fetchall()
        cursor.close()
        conn.close()
        return {"success": True, "result": rows}
    except Exception as e:
        return {"success": False, "error": str(e)}

# Основная функция для обработки одного вопроса
def process_single_question(question, table_schema, api_key, top_k=5):
    try:
        # Поиск релевантных SQL-запросов
        relevant_queries = find_relevant_queries(question, top_k)

        # Генерация SQL-запроса с помощью LLM
        model_response = generate_sql_code(question, table_schema, relevant_queries, api_key)

        # Выполнение SQL-запроса в MySQL
        test_result = execute_mysql_query(model_response)

        # Возвращаем результаты
        return {
            "question": question,
            "generated_query": model_response,
            "execution_result": test_result,
            "relevant_queries": relevant_queries
        }

    except Exception as e:
        print(f"Ошибка при обработке вопроса '{question}': {e}")
        return {
            "question": question,
            "error": str(e)
        }


In [ ]:
# Обработка одного вопроса
question = test_questions[6]  # Берём первый вопрос из test_questions
result = process_single_question(question, table_schema, OPENROUTER_API_KEY)

# Вывод результата
print("Вопрос:", result["question"])
if "error" in result:
    print("Ошибка:", result["error"])
else:
    print("Сгенерированный SQL-запрос:", result["generated_query"])
    print("Результат выполнения запроса:", result["execution_result"])
    print("Релевантные SQL-запросы:", result["relevant_queries"])

Вопрос: i need to fly from st. louis to milwaukee on wednesday afternoon
Сгенерированный SQL-запрос: ```sql
SELECT DISTINCT flight_1.flight_id
FROM flight flight_1
JOIN airport_service airport_service_1 ON flight_1.from_airport = airport_service_1.airport_code
JOIN city city_1 ON airport_service_1.city_code = city_1.city_code
JOIN airport_service airport_service_2 ON flight_1.to_airport = airport_service_2.airport_code
JOIN city city_2 ON airport_service_2.city_code = city_2.city_code
JOIN days days_1 ON flight_1.flight_days = days_1.days_code
JOIN date_day date_day_1 ON days_1.day_name = date_day_1.day_name
WHERE city_1.city_name = 'ST. LOUIS'
  AND city_2.city_name = 'MILWAUKEE'
  AND date_day_1.day_name = 'WEDNESDAY'
  AND flight_1.departure_time BETWEEN '1200' AND '2359';
```
Результат выполнения запроса: {'success': False, 'error': '(2003, "Can\'t connect to MySQL server on \'localhost\' ([WinError 10061] Подключение не установлено, т.к. конечный компьютер отверг запрос на подключ

`(╥﹏╥)` Ну тут у меня беда локально с mysql случилась `(╥﹏╥)`

In [29]:
question = test_questions[3]  # Берём первый вопрос из test_questions
result = process_single_question(question, table_schema, OPENROUTER_API_KEY)

In [ ]:
# Функция для вычисления метрик
def calculate_metrics(true_query, generated_query):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = scorer.score(true_query, generated_query)

    true_tokens = true_query.split()
    generated_tokens = generated_query.split()
    smooth_fn = SmoothingFunction().method1
    bleu_score = sentence_bleu([true_tokens], generated_tokens, smoothing_function=smooth_fn)

    meteor = meteor_score([true_query.split()], generated_query.split())

    return {
        "BLEU": bleu_score,
        "METEOR": meteor,
        "ROUGE-1": rouge_scores['rouge1'].fmeasure,
        "ROUGE-2": rouge_scores['rouge2'].fmeasure,
        "ROUGE-L": rouge_scores['rougeL'].fmeasure
    }

# Функция для оценки модели на первых N вопросах
def evaluate_model(test_questions, test_queries, table_schema, api_key, top_k=3, num_questions=10):
    metrics_list = []
    successful_queries = 0

    for i in range(min(num_questions, len(test_questions))):  # Обрабатываем первые N вопросов
        question = test_questions[i]
        true_query = test_queries[i]

        try:
            # Генерация SQL-запроса и выполнение
            result = process_single_question(question, table_schema, api_key, top_k)

            if "error" in result:
                print(f"Ошибка при обработке вопроса {i + 1}: {result['error']}")
                continue

            # Вычисление метрик
            metrics = calculate_metrics(true_query, result["generated_query"])
            metrics_list.append(metrics)

            # Проверка успешности выполнения запроса
            if result["execution_result"]["success"]:
                successful_queries += 1

        except Exception as e:
            print(f"Ошибка при обработке вопроса {i + 1}: {e}")

    # Усреднение метрик
    avg_metrics = {
        "BLEU": sum(m["BLEU"] for m in metrics_list) / len(metrics_list),
        "METEOR": sum(m["METEOR"] for m in metrics_list) / len(metrics_list),
        "ROUGE-1": sum(m["ROUGE-1"] for m in metrics_list) / len(metrics_list),
        "ROUGE-2": sum(m["ROUGE-2"] for m in metrics_list) / len(metrics_list),
        "ROUGE-L": sum(m["ROUGE-L"] for m in metrics_list) / len(metrics_list),
    }

    # Вывод результатов
    print(f"Оценка на первых {num_questions} вопросах:")
    print(f"Средний BLEU: {avg_metrics['BLEU']:.4f}")
    print(f"Средний METEOR: {avg_metrics['METEOR']:.4f}")
    print(f"Средний ROUGE-1: {avg_metrics['ROUGE-1']:.4f}")
    print(f"Средний ROUGE-2: {avg_metrics['ROUGE-2']:.4f}")
    print(f"Средний ROUGE-L: {avg_metrics['ROUGE-L']:.4f}")
    print(f"Количество успешных запросов: {successful_queries} из {num_questions}")

    return avg_metrics, successful_queries


In [15]:
# Оценка модели на первых 10 вопросах
avg_metrics, successful_queries = evaluate_model(test_questions, test_queries, table_schema, OPENROUTER_API_KEY, num_questions=10)

Оценка на первых 10 вопросах:
Средний BLEU: 0.0256
Средний METEOR: 0.1887
Средний ROUGE-1: 0.6421
Средний ROUGE-2: 0.3910
Средний ROUGE-L: 0.5242
Количество успешных запросов: 0 из 10
